> Summarized from this awesome video about the discussion of chains and agents: https://www.youtube.com/watch?v=bYLHklxEd_k&t=1801s


- Chains and agents represent two concepts in technology with distinct functionalities. Chains refer to linear sequences of events or processes that rely on the step-by-step execution of tasks, often requiring predefined inputs and producing predictable outcomes. 

- Agents, conversely, are autonomous systems capable of making decisions and performing tasks independently, adapting to new situations without explicit instructions. While chains excel in structured environments with clear rules, agents are designed to navigate complex, dynamic scenarios, learning and adjusting their behavior over time. The core difference lies in the level of autonomy and complexity each can handle, with agents typically seen as more advanced due to their self-governing nature.

In [26]:
# !pip install langchain
# !pip install langchain-openai
# !pip install langchainhub
# !pip install pypdf
# !pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.1 MB/s eta 0:00:000:00:01m eta 0:00:01


Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

In [45]:
# import os

# # Set OPENAI API Key

# os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)

# from dotenv import load_dotenv
# load_dotenv("./.env")

True

![](2023-10-27-19-28-10.png)

In [47]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [48]:
search = TavilySearchResults()

In [49]:
search.invoke("what is the weather in Portugal?")

"HTTPError('502 Server Error: Bad Gateway for url: https://api.tavily.com/search')"

In [50]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [51]:
retriever.get_relevant_documents("What is langchain?")[0]

Document(page_content='Introduction | 🦜️🔗 Langchain', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'})

In [52]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [53]:
tools = [search, retriever_tool]

In [54]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?')

In [55]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [56]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [57]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [58]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [59]:
agent_executor.invoke({"input": "What are some useful applications you can build with LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `langchain_search` with `{'query': 'useful applications of LangChain'}`


Write your applications in LangChain/LangChain.js. Hit the ground running using Templates for reference.Productionize: Use LangSmith to inspect, test and monitor your chains, so that you can constantly improve and deploy with confidence.Deploy: Turn any chain into an API with LangServe.LangChain Libraries​The main value props of the LangChain packages are:Components: composable tools and integrations for working with language models. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or notOff-the-shelf chains: built-in assemblages of components for accomplishing higher-level tasksOff-the-shelf chains make it easy to get started. Components make it easy to customize existing chains and build new ones.The LangChain libraries themselves are made up of several different packages.langchain-core: Base abstractions and Lan

{'input': 'What are some useful applications you can build with LangChain?',
 'output': 'LangChain can be used to build a variety of useful applications, including:\n\n1. Context-aware applications: LangChain enables applications that are context-aware by connecting a language model to sources of context such as prompt instructions, few-shot examples, and content to ground its response in.\n\n2. Reasoning applications: LangChain allows applications to rely on a language model to reason about how to answer based on provided context and what actions to take.\n\n3. Components and off-the-shelf chains: LangChain provides composable tools and integrations for working with language models, as well as built-in assemblages of components for accomplishing higher-level tasks. These components and off-the-shelf chains make it easy to customize existing chains and build new ones.\n\n4. Deployment as REST API: LangChain can turn any chain into an API with LangServe, making it easy to deploy applica